In [2]:
import json
import pandas as pd

## 1 - Reconstruction des données EIREL

In [3]:
# Chargement des données avec les bonnes colonnes
def load_eirel(filepath, config_path='data/bronze/reference/eirel_columns.json'):
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    colonnes = {int(k): v for k, v in config['fixed_columns'].items()}
    
    temp = pd.read_csv(filepath, sep=";", skiprows=2, header=None, usecols=[17], nrows=1)
    nb_listes = temp.iloc[0, 0]
    
    start_idx = config['dynamic_columns']['start_index']
    for i in range(1, nb_listes + 1):
        colonnes[start_idx + 2*(i-1)] = f"code_liste_{i}"
        colonnes[start_idx + 2*(i-1) + 1] = f"voix_liste_{i}"
    
    data = pd.read_csv(filepath, sep=";", skiprows=2, header=None)
    data.rename(columns=colonnes, inplace=True)
    
    return data

In [4]:
# Récupération des données EIREL avec les configurations précices
data = load_eirel("data/bronze/eirel/TEST_EIREL_MEUDON_28052024.txt")
data.head()

,num_ligne,type_election,annee,tour,departement,commune,bureau_vote,circonscription,canton,indicateur,...,code_liste_34,voix_liste_34,code_liste_35,voix_liste_35,code_liste_36,voix_liste_36,code_liste_37,voix_liste_37,code_liste_38,voix_liste_38
0,1,ER,2024,1,92,48,1,17,8,I,...,8,1,32,1,44,10,36,1,46,2
1,2,ER,2024,1,92,48,2,17,8,I,...,8,1,32,20,44,1,36,2,46,1
2,3,ER,2024,1,92,48,3,17,8,I,...,8,16,32,12,44,12,36,1,46,1
3,4,ER,2024,1,92,48,4,17,8,I,...,8,0,32,0,44,0,36,0,46,0
4,5,ER,2024,1,92,48,5,17,8,I,...,8,6,32,15,44,3,36,15,46,23


In [5]:
# Aperçu des colonnes
data.columns

Index(['num_ligne', 'type_election', 'annee', 'tour', 'departement', 'commune',
       'bureau_vote', 'circonscription', 'canton', 'indicateur', 'nb_inscrits',
       'nb_abstentions', 'nb_votants', 'nb_emargements', 'nb_blanc', 'nb_nuls',
       'nb_exprimes', 'nb_listes', 'code_liste_1', 'voix_liste_1',
       'code_liste_2', 'voix_liste_2', 'code_liste_3', 'voix_liste_3',
       'code_liste_4', 'voix_liste_4', 'code_liste_5', 'voix_liste_5',
       'code_liste_6', 'voix_liste_6', 'code_liste_7', 'voix_liste_7',
       'code_liste_8', 'voix_liste_8', 'code_liste_9', 'voix_liste_9',
       'code_liste_10', 'voix_liste_10', 'code_liste_11', 'voix_liste_11',
       'code_liste_12', 'voix_liste_12', 'code_liste_13', 'voix_liste_13',
       'code_liste_14', 'voix_liste_14', 'code_liste_15', 'voix_liste_15',
       'code_liste_16', 'voix_liste_16', 'code_liste_17', 'voix_liste_17',
       'code_liste_18', 'voix_liste_18', 'code_liste_19', 'voix_liste_19',
       'code_liste_20', 'voix_list

In [6]:
# Dimension du jeu de données
data.shape

(32, 94)

Réflexion sur le modèle de données : 
- NUMERO_BUREAU --> Dispo "bureau_vote" 
- CODE_LIEU_DE_VOTE --> Dispo "bureau_vote"
- NOM_BUREAU --> Jointure "bureau_vote"
- circos --> canton
- nb_Inscrits --> nb_inscrits
- nb_Emargements --> emargements
- nb_Votants --> votants
- nb_Blanc --> nb_blanc
- nb_Nuls --> nb_nuls
- nb_Exprimes --> ??? 
- nb_Procurations --> ??? 
- Candidat --> Pivot table
- nb_voix --> Pivot table


Intéressant à récupérer pour interoperabilité et plus : 
- type_election
- annee
- tour
- commune
- abstentions

## 2 - Sélection des colonnes à intégrer pour constituer le jeu de données

In [7]:
with open("data/bronze/reference/columns_repartition.json") as config_cols:
    cols_used = json.load(config_cols)

In [8]:
data[cols_used["metrics_bdv"]].head()

,bureau_vote,type_election,annee,tour,commune,canton,nb_inscrits,nb_abstentions,nb_votants,nb_emargements,nb_blanc,nb_nuls
0,1,ER,2024,1,48,8,939,339,600,600,20,10
1,2,ER,2024,1,48,8,1007,507,500,500,2,2
2,3,ER,2024,1,48,8,1023,223,800,800,5,2
3,4,ER,2024,1,48,8,1066,266,800,800,0,0
4,5,ER,2024,1,48,8,1020,270,750,750,2,2


In [9]:
data[cols_used["candidats_voix"]].head()

,bureau_vote,code_liste_1,voix_liste_1,code_liste_2,voix_liste_2,code_liste_3,voix_liste_3,code_liste_4,voix_liste_4,code_liste_5,...,code_liste_34,voix_liste_34,code_liste_35,voix_liste_35,code_liste_36,voix_liste_36,code_liste_37,voix_liste_37,code_liste_38,voix_liste_38
0,1,38,10,40,20,28,10,25,5,5,...,8,1,32,1,44,10,36,1,46,2
1,2,38,2,40,2,28,3,25,5,5,...,8,1,32,20,44,1,36,2,46,1
2,3,38,50,40,20,28,20,25,70,5,...,8,16,32,12,44,12,36,1,46,1
3,4,38,10,40,10,28,20,25,130,5,...,8,0,32,0,44,0,36,0,46,0
4,5,38,6,40,2,28,10,25,58,5,...,8,6,32,15,44,3,36,15,46,23


## 3 - Transformation de données
> Faire une jointure pour récupérer les noms de bureaux de vote

### 3.1 - Récupération des noms de bureau

In [10]:
# Données d'enrichissement --> 
bdv = pd.read_csv(
    "https://data.meudon.fr/api/explore/v2.1/catalog/datasets/bureaux-de-vote-electoraux-2026/exports/csv", 
    sep=";",
    usecols=["num_bureau", "nom_bureau"]
)
bdv.head()

,num_bureau,nom_bureau
0,19,Ecole maternelle La Ruche
1,17,Ecole maternelle Prévert
2,32,L'Avant-Seine
3,16,Ecole maternelle Jean de la Fontaine
4,20,Pôle intergénérations


In [11]:
nom_bureau = data.merge(
    bdv, 
    left_on="bureau_vote", 
    right_on="num_bureau", 
    how="left"
)
nom_bureau = nom_bureau[["bureau_vote", "nom_bureau"]].copy()
nom_bureau.head()

,bureau_vote,nom_bureau
0,1,Mairie
1,2,Club Séniors Lavoisier
2,3,Espace jeunesse Val Fleury
3,4,Complexe sportif René Leduc
4,5,Ecole élémentaire Jules Ferry


### 3.2 - Transformation pivot table pour récupération des candidats et nombres de voix

In [12]:
candidats_voix = data[cols_used["candidats_voix"]].head()
candidats_voix.head()

,bureau_vote,code_liste_1,voix_liste_1,code_liste_2,voix_liste_2,code_liste_3,voix_liste_3,code_liste_4,voix_liste_4,code_liste_5,...,code_liste_34,voix_liste_34,code_liste_35,voix_liste_35,code_liste_36,voix_liste_36,code_liste_37,voix_liste_37,code_liste_38,voix_liste_38
0,1,38,10,40,20,28,10,25,5,5,...,8,1,32,1,44,10,36,1,46,2
1,2,38,2,40,2,28,3,25,5,5,...,8,1,32,20,44,1,36,2,46,1
2,3,38,50,40,20,28,20,25,70,5,...,8,16,32,12,44,12,36,1,46,1
3,4,38,10,40,10,28,20,25,130,5,...,8,0,32,0,44,0,36,0,46,0
4,5,38,6,40,2,28,10,25,58,5,...,8,6,32,15,44,3,36,15,46,23


In [13]:
# Sélectionner uniquement les colonnes voix_liste_*
voix_cols = [col for col in candidats_voix.columns if col.startswith('voix_liste_')]

candidats_voix_long = candidats_voix.melt(
    id_vars='bureau_vote',
    value_vars=voix_cols,
    var_name='candidat',
    value_name='nb_voix'
)

# Extraire le numéro du candidat
# candidats_voix_long['candidat'] = candidats_voix_long['candidat'].str.extract(r'(\d+)').astype(int)
candidats_voix_long['candidat'] = candidats_voix_long['candidat']

candidats_voix_long.head()

,bureau_vote,candidat,nb_voix
0,1,voix_liste_1,10
1,2,voix_liste_1,2
2,3,voix_liste_1,50
3,4,voix_liste_1,10
4,5,voix_liste_1,6


In [14]:
candidats_voix_long.shape

(190, 3)

In [15]:
# Vérification du nombre de candidats
candidats_voix_long.candidat.nunique()

38

### 3.3 - Jointure finale

In [16]:
data[cols_used["metrics_bdv"]].head()

,bureau_vote,type_election,annee,tour,commune,canton,nb_inscrits,nb_abstentions,nb_votants,nb_emargements,nb_blanc,nb_nuls
0,1,ER,2024,1,48,8,939,339,600,600,20,10
1,2,ER,2024,1,48,8,1007,507,500,500,2,2
2,3,ER,2024,1,48,8,1023,223,800,800,5,2
3,4,ER,2024,1,48,8,1066,266,800,800,0,0
4,5,ER,2024,1,48,8,1020,270,750,750,2,2


In [17]:
data_kpi = data[cols_used["metrics_bdv"]].copy()

In [18]:
data_gold = data_kpi.merge(
    candidats_voix_long, 
    left_on="bureau_vote", 
    right_on="bureau_vote", 
    how="inner"
)

In [19]:
data_gold.head()

,bureau_vote,type_election,annee,tour,commune,canton,nb_inscrits,nb_abstentions,nb_votants,nb_emargements,nb_blanc,nb_nuls,candidat,nb_voix
0,1,ER,2024,1,48,8,939,339,600,600,20,10,voix_liste_1,10
1,1,ER,2024,1,48,8,939,339,600,600,20,10,voix_liste_2,20
2,1,ER,2024,1,48,8,939,339,600,600,20,10,voix_liste_3,10
3,1,ER,2024,1,48,8,939,339,600,600,20,10,voix_liste_4,5
4,1,ER,2024,1,48,8,939,339,600,600,20,10,voix_liste_5,2


In [22]:
data_gold[data_gold["bureau_vote"] == 2]

,bureau_vote,type_election,annee,tour,commune,canton,nb_inscrits,nb_abstentions,nb_votants,nb_emargements,nb_blanc,nb_nuls,candidat,nb_voix
38,2,ER,2024,1,48,8,1007,507,500,500,2,2,voix_liste_1,2
39,2,ER,2024,1,48,8,1007,507,500,500,2,2,voix_liste_2,2
40,2,ER,2024,1,48,8,1007,507,500,500,2,2,voix_liste_3,3
41,2,ER,2024,1,48,8,1007,507,500,500,2,2,voix_liste_4,5
42,2,ER,2024,1,48,8,1007,507,500,500,2,2,voix_liste_5,6
43,2,ER,2024,1,48,8,1007,507,500,500,2,2,voix_liste_6,10
44,2,ER,2024,1,48,8,1007,507,500,500,2,2,voix_liste_7,2
45,2,ER,2024,1,48,8,1007,507,500,500,2,2,voix_liste_8,10
46,2,ER,2024,1,48,8,1007,507,500,500,2,2,voix_liste_9,20
47,2,ER,2024,1,48,8,1007,507,500,500,2,2,voix_liste_10,2
